# Indexing using ElasticSearch

## Install and Import necessary packages and libraries

In [8]:
# %pip install elasticsearch

In [1]:
import os
from elasticsearch import Elasticsearch
import re

## Configurations

In [7]:
host = 'http://150.239.171.68'
port = 9200
user = 'elastic'
password = '5uaaIyEEWrqtjT*Pg4JZ' #password for elastic user - setup by amit
index_name = "test_index" 

# Directory containing text files
directory = './ibm_developer_metadata/'

#define index mapping
mapping = {
    "mappings": {
        "properties": {
            "content": {
              "type": "text"
            },
#             "url" :{
#                 "type" : "text"
#             }
          }
    }
}

# es = Elasticsearch([{'host': host, 'port': port, "scheme": "http"}],verify_certs=False,basic_auth=[user, password])
es = Elasticsearch(['http://150.239.171.68:9200'])


## Create a connection to Elasticsearch

In [8]:

# Create the index
try:
    response = es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created successfully.")
except:
    print(f"Failed to create index '{index_name}'.")
    
# Get the count of documents in the index
document_count = es.count(index=index_name)['count']
print(f"Number of documents before adding documents '{index_name}': {document_count}")

Failed to create index 'test_index'.


/var/folders/gy/6lyfn5752472j2htx4g01gc40000gn/T/ipykernel_93785/2853410201.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.indices.create(index=index_name, body=mapping)


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fbe6b0bb520>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known))

## Adding documents to selected index

In [9]:
# Iterate over text files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        
        # Read the contents of the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            print(file)
            pattern_url = r'^URL:.*$'
            matches_url = re.findall(pattern_url, content, re.MULTILINE)
            
            
            print("****")
            matches_content = content.split("content:")[1]
            #print(matches_content)
            

        # Create a document to index
        document = {
            'file_name': filename,
            'content': matches_content,
            'url' :"https://developer.ibm.com"+(matches_url[0]).split("URL:")[1]
        }

        # Index the document in Elasticsearch
        es.index(index=index_name, body=document)
        print(f"Indexed document: {filename}")
        

print("Indexing completed.")
# Get the count of documents in the index
document_count = es.count(index=index_name)['count']
print(f"Number of documents after adding documents '{index_name}': {document_count}")

FileNotFoundError: [Errno 2] No such file or directory: './ibm_developer_metadata/'

## View sample docs in an index



In [11]:
index_name = "test_index_2"


# Define the search query to retrieve sample documents
search_query = {
    "query": {
            "match": {
      "file_name": "What is IBM Watson?"
    }
    }
}

# Perform the search query
response = es.search(index=index_name, body=search_query)
# Extract the sample documents from the search response
sample_documents = [hit["_source"] for hit in response["hits"]["hits"]]
sample_documents

/var/folders/gy/6lyfn5752472j2htx4g01gc40000gn/T/ipykernel_93785/3926384110.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fbe69a270d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known))

In [20]:
for doc in sample_documents:
    print(doc["file_name"])

What is IBM LinuxONE III?.txt
What is Messaging?.txt
What is Apache Kafka?.txt
What is edge computing?.txt
What is DevSecOps and why is it so important?.txt
What is big data? More than volume, velocity and variety.txt
What is cURL and how does it relate to APIs?.txt
Introduction to IBM Watson Studio.txt
AIOps, what and why.txt
Sentiment analysis using IBM Watson NLP.txt


## Delete documents in an index

In [10]:
index_name = 'test_index'

# Define the delete query to match all documents
delete_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the delete by query operation
response = es.delete_by_query(index=index_name, body=delete_query)

# Print the response
print(response)

/tmp/ipykernel_4107670/891584841.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.delete_by_query(index=index_name, body=delete_query)
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'took': 408, 'timed_out': False, 'total': 136, 'deleted': 136, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


In [21]:
document_count = es.count(index=index_name)['count']
document_count

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1472